In [617]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import requests                    # for "get" request to API
import json                        # parse json into a list
import datetime as dt 
from matplotlib import rcParams


from matplotlib import rcParams
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from csv import writer

In [618]:
def get_binance_bars(coin, base_currency, interval, startTime, endTime):
    ''' 
    download the data of the coins and store as .csv file 
    '''
     
    symbol = str(coin+base_currency)
    
    # In case of HTTP 5xx Error, or data is not fetched from the server, Try changing 'api1' in the URL to 'api2' or 'api3'

    url = "https://api1.binance.com/api/v3/klines"
 
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '4000'
 
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}
    
    data = json.loads(requests.get(url, params = req_params).text)

    if isinstance(data , dict):
      print(f"Returned Invalid Code Error for Coin {coin}")
      return None
    
    else:
      if not data:
        print(f"Did not receieve OK response from Coinbase API for Coin {coin}")
        return None
      
      else:
        df = pd.DataFrame(data)

        df = df.iloc[:, 0:6]
        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume' ]
    
        df.open      = df.open.astype("float")
        df.high      = df.high.astype("float")
        df.low       = df.low.astype("float")
        df.close     = df.close.astype("float")
        df.volume    = df.volume.astype("float")
         

        df.set_index('datetime', inplace= True)
 
        df.index = [pd.Timestamp(x/1000, unit='s', tz= 'Asia/Kolkata') for x in df.index]
        df = df.astype('float')

        # df['datetime'] = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
        # df.set_index('datetime', inplace= True)  
        

        df.to_csv(f'{coin}.csv')

In [619]:
# KEY PARAMETERS
# end_date =  dt.datetime.today()
# start_date = end_date - dt.timedelta(hours= 996)
# interval = '1h'  
start_date = dt.datetime(2022, 5, 30)
end_date =  dt.datetime.today()
# end_date =  dt.datetime(2022, 5, 22)
interval = '1h' 
base_currency = 'BUSD'

coins = ['BNB']

# Downloading the CSV files for the coins
for coin in coins:
  get_binance_bars(coin, base_currency, interval, start_date, end_date)

In [620]:
#   1 hour data
data_bnb_1h_live = pd.read_csv('BNB.csv')
data_bnb_1h_live = data_bnb_1h_live.rename(columns={'Unnamed: 0': 'datetime' })
data_bnb_1h_live.set_index('datetime', inplace= True)

# print( "live  data", data_bnb_1h.shape)
data_bnb_1h_live =  data_bnb_1h_live.iloc[19:]
data_bnb_1h_live 

,open,high,low,close,volume
datetime,,,,,
2022-05-31 00:30:00+05:30,313.7,315.3,312.1,313.2,8557.074
2022-05-31 01:30:00+05:30,313.1,318.7,311.8,318.6,22546.296
2022-05-31 02:30:00+05:30,318.6,322.8,317.5,321.6,28327.154
2022-05-31 03:30:00+05:30,321.7,322.5,319.4,321.8,17759.263
2022-05-31 04:30:00+05:30,321.8,323.8,320.3,321.5,14876.252
2022-05-31 05:30:00+05:30,321.6,324.5,319.6,320.8,17959.655
2022-05-31 06:30:00+05:30,320.7,321.3,317.5,319.6,9991.861
2022-05-31 07:30:00+05:30,319.7,321.7,318.9,321.6,12779.822
2022-05-31 08:30:00+05:30,321.6,322.5,320.0,320.6,10713.522


In [621]:
past_1h = pd.read_csv( 'data_bnb_1h.csv')
past_1h.set_index('datetime', inplace= True)
past_1h.tail()

,open,high,low,close,volume
datetime,,,,,
2022-05-30 20:00:00,313.1,318.7,311.8,318.6,22546.296
2022-05-30 21:00:00,318.6,322.8,317.5,321.6,28327.154
2022-05-30 22:00:00,321.7,322.5,319.4,321.8,17759.263
2022-05-30 23:00:00,321.8,323.8,320.3,321.5,14876.252
2022-05-31 00:00:00,321.6,324.5,319.6,320.8,17959.655


In [622]:
frames1 = [past_1h,data_bnb_1h_live]
data_bnb_1h  = pd.concat(frames1)
print(data_bnb_1h )


                               open      high       low     close     volume
datetime                                                                    
2019-09-19 10:00:00         21.2021   21.2058   20.7415   20.7415     24.790
2019-09-19 11:00:00         20.9594   20.9594   20.9594   20.9594      1.020
2019-09-19 12:00:00         20.9454   21.0119   20.9331   20.9970    658.150
2019-09-19 13:00:00         20.9970   20.9970   20.9970   20.9970      0.000
2019-09-19 14:00:00         20.9970   20.9970   20.9970   20.9970      0.000
...                             ...       ...       ...       ...        ...
2022-05-31 10:30:00+05:30  318.2000  320.3000  317.2000  318.5000  11641.580
2022-05-31 11:30:00+05:30  318.5000  319.0000  315.7000  317.3000  12718.784
2022-05-31 12:30:00+05:30  317.3000  318.7000  316.8000  318.2000   8004.377
2022-05-31 13:30:00+05:30  318.2000  321.2000  317.8000  319.9000   9420.093
2022-05-31 14:30:00+05:30  319.9000  320.1000  318.1000  319.8000   5534.628

In [623]:
def get_binance_bars(coin, base_currency, interval, startTime, endTime):
    ''' 
    download the data of the coins and store as .csv file 
    '''
     
    symbol = str(coin+base_currency)
    
    # In case of HTTP 5xx Error, or data is not fetched from the server, Try changing 'api1' in the URL to 'api2' or 'api3'

    url = "https://api1.binance.com/api/v3/klines"
 
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '4000'
 
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}
    
    data = json.loads(requests.get(url, params = req_params).text)

    if isinstance(data , dict):
      print(f"Returned Invalid Code Error for Coin {coin}")
      return None
    
    else:
      if not data:
        print(f"Did not receieve OK response from Coinbase API for Coin {coin}")
        return None
      
      else:
        df = pd.DataFrame(data)

        df = df.iloc[:, 0:6]
        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume' ]
    
        df.open      = df.open.astype("float")
        df.high      = df.high.astype("float")
        df.low       = df.low.astype("float")
        df.close     = df.close.astype("float")
        df.volume    = df.volume.astype("float")
         

        df.set_index('datetime', inplace= True)
 
        df.index = [pd.Timestamp(x/1000, unit='s', tz= 'Asia/Kolkata') for x in df.index]
        df = df.astype('float')

        # df['datetime'] = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
        # df.set_index('datetime', inplace= True)  
        

        df.to_csv(f'{coin}.csv')

In [624]:

# KEY PARAMETERS
# end_date =  dt.datetime.today()
# start_date = end_date - dt.timedelta(hours= 1993)
# interval = '2h'  
start_date = dt.datetime(2022, 5, 30)
end_date =  dt.datetime.today()
# end_date =  dt.datetime(2022, 5, 22)
interval = '2h' 
base_currency = 'BUSD'

coins = ['BNB']

# Downloading the CSV files for the coins
for coin in coins:
  get_binance_bars(coin, base_currency, interval, start_date, end_date)

In [625]:
#   2 hours data
data_bnb_2h_live = pd.read_csv('BNB.csv')
data_bnb_2h_live = data_bnb_2h_live.rename(columns={'Unnamed: 0': 'datetime' })
data_bnb_2h_live.set_index('datetime', inplace= True)

# print( "live  data", data_bnb_2h.shape)
data_bnb_2h_live = data_bnb_2h_live.iloc[19:]
data_bnb_2h_live 

,open,high,low,close,volume
datetime,,,,,


In [626]:
past_2h = pd.read_csv( 'data_bnb_2h.csv')
past_2h.set_index('datetime', inplace= True)
past_2h.tail()

,open,high,low,close,volume
datetime,,,,,
2022-05-30 16:00:00,313.5,315.4,311.9,314.5,19288.914
2022-05-30 18:00:00,314.4,315.3,312.1,313.2,15060.997
2022-05-30 20:00:00,313.1,322.8,311.8,321.6,50873.450
2022-05-30 22:00:00,321.7,323.8,319.4,321.5,32635.515
2022-05-31 00:00:00,321.6,324.5,317.5,319.6,27951.516


In [627]:
frames2 = [past_2h,data_bnb_2h_live]
data_bnb_2h  = pd.concat(frames2)
print(data_bnb_2h  )


                         open      high       low     close     volume
datetime                                                              
2019-09-19 10:00:00   21.2021   21.2058   20.7415   20.9594     25.810
2019-09-19 12:00:00   20.9454   21.0119   20.9331   20.9970    658.150
2019-09-19 14:00:00   20.9042   20.9722   20.8856   20.9324    134.700
2019-09-19 16:00:00   20.9269   21.4061   20.9269   21.3955    603.730
2019-09-19 18:00:00   21.3647   21.3915   21.1456   21.2961    442.530
...                       ...       ...       ...       ...        ...
2022-05-30 16:00:00  313.5000  315.4000  311.9000  314.5000  19288.914
2022-05-30 18:00:00  314.4000  315.3000  312.1000  313.2000  15060.997
2022-05-30 20:00:00  313.1000  322.8000  311.8000  321.6000  50873.450
2022-05-30 22:00:00  321.7000  323.8000  319.4000  321.5000  32635.515
2022-05-31 00:00:00  321.6000  324.5000  317.5000  319.6000  27951.516

[11805 rows x 5 columns]


In [628]:
def get_binance_bars(coin, base_currency, interval, startTime, endTime):
    ''' 
    download the data of the coins and store as .csv file 
    '''
     
    symbol = str(coin+base_currency)
    
    # In case of HTTP 5xx Error, or data is not fetched from the server, Try changing 'api1' in the URL to 'api2' or 'api3'

    url = "https://api1.binance.com/api/v3/klines"
 
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '4000'
 
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}
    
    data = json.loads(requests.get(url, params = req_params).text)

    if isinstance(data , dict):
      print(f"Returned Invalid Code Error for Coin {coin}")
      return None
    
    else:
      if not data:
        print(f"Did not receieve OK response from Coinbase API for Coin {coin}")
        return None
      
      else:
        df = pd.DataFrame(data)

        df = df.iloc[:, 0:6]
        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume' ]
    
        df.open      = df.open.astype("float")
        df.high      = df.high.astype("float")
        df.low       = df.low.astype("float")
        df.close     = df.close.astype("float")
        df.volume    = df.volume.astype("float")
         

        df.set_index('datetime', inplace= True)
 
        df.index = [pd.Timestamp(x/1000, unit='s', tz= 'Asia/Kolkata') for x in df.index]
        df = df.astype('float')

        # df['datetime'] = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
        # df.set_index('datetime', inplace= True)  
        

        df.to_csv(f'{coin}.csv')

In [629]:
# KEY PARAMETERS
# end_date =  dt.datetime.today()
# start_date = end_date - dt.timedelta(hours= 3986)
# interval = '4h'  
start_date = dt.datetime(2022, 5, 30)
end_date =  dt.datetime.today()
# end_date =  dt.datetime(2022, 5, 22)
interval = '4h' 
base_currency = 'BUSD'

coins = ['BNB']

# Downloading the CSV files for the coins
for coin in coins:
  get_binance_bars(coin, base_currency, interval, start_date, end_date)

In [630]:
#4 hours data
data_bnb_4h_live = pd.read_csv('BNB.csv')
data_bnb_4h_live = data_bnb_4h_live.rename(columns={'Unnamed: 0': 'datetime' })
data_bnb_4h_live.set_index('datetime', inplace= True)

# print( "live  data", data_bnb_4h.shape)
data_bnb_4h_live = data_bnb_4h_live.iloc[4:]
data_bnb_4h_live.head()

,open,high,low,close,volume
datetime,,,,,
2022-05-30 21:30:00+05:30,313.5,315.4,311.9,313.2,34349.911
2022-05-31 01:30:00+05:30,313.1,323.8,311.8,321.5,83508.965
2022-05-31 05:30:00+05:30,321.6,324.5,317.5,320.6,51444.860
2022-05-31 09:30:00+05:30,320.6,320.9,315.7,318.2,44442.241
2022-05-31 13:30:00+05:30,318.2,321.2,317.8,319.8,14954.721


In [631]:

past_4h = pd.read_csv( 'data_bnb_4h.csv')
past_4h.set_index('datetime', inplace= True)
past_4h 

,open,high,low,close,volume
datetime,,,,,
2019-09-19 08:00:00,21.2021,21.2058,20.7415,20.9594,25.810
2019-09-19 12:00:00,20.9454,21.0119,20.8856,20.9324,792.850
2019-09-19 16:00:00,20.9269,21.4061,20.9269,21.2961,1046.260
2019-09-19 20:00:00,21.2960,21.5998,21.1872,21.5998,551.800
2019-09-20 00:00:00,21.4918,21.4918,21.3444,21.3579,95.940
...,...,...,...,...,...
2022-05-30 08:00:00,319.3000,320.1000,314.7000,317.0000,46478.174
2022-05-30 12:00:00,317.0000,319.3000,311.5000,313.4000,83017.971
2022-05-30 16:00:00,313.5000,315.4000,311.9000,313.2000,34349.911


In [632]:
frames3 = [past_4h,data_bnb_4h_live]
data_bnb_4h  = pd.concat(frames3)
print(data_bnb_4h.tail( ) )


                            open   high    low  close     volume
datetime                                                        
2022-05-30 21:30:00+05:30  313.5  315.4  311.9  313.2  34349.911
2022-05-31 01:30:00+05:30  313.1  323.8  311.8  321.5  83508.965
2022-05-31 05:30:00+05:30  321.6  324.5  317.5  320.6  51444.860
2022-05-31 09:30:00+05:30  320.6  320.9  315.7  318.2  44442.241
2022-05-31 13:30:00+05:30  318.2  321.2  317.8  319.8  14954.721


In [633]:
def get_binance_bars(coin, base_currency, interval, startTime, endTime):
    ''' 
    download the data of the coins and store as .csv file 
    '''
     
    symbol = str(coin+base_currency)
    
    # In case of HTTP 5xx Error, or data is not fetched from the server, Try changing 'api1' in the URL to 'api2' or 'api3'

    url = "https://api1.binance.com/api/v3/klines"
 
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '4000'
 
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}
    
    data = json.loads(requests.get(url, params = req_params).text)

    if isinstance(data , dict):
      print(f"Returned Invalid Code Error for Coin {coin}")
      return None
    
    else:
      if not data:
        print(f"Did not receieve OK response from Coinbase API for Coin {coin}")
        return None
      
      else:
        df = pd.DataFrame(data)

        df = df.iloc[:, 0:6]
        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume' ]
    
        df.open      = df.open.astype("float")
        df.high      = df.high.astype("float")
        df.low       = df.low.astype("float")
        df.close     = df.close.astype("float")
        df.volume    = df.volume.astype("float")
         

        df.set_index('datetime', inplace= True)
 
        df.index = [pd.Timestamp(x/1000, unit='s', tz= 'Asia/Kolkata') for x in df.index]
        df = df.astype('float')

        # df['datetime'] = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
        # df.set_index('datetime', inplace= True)  
        

        df.to_csv(f'{coin}.csv')

In [634]:
# KEY PARAMETERS
# end_date =  dt.datetime.today()
# start_date = end_date - dt.timedelta(hours= 11950)
# interval = '12h'  
start_date = dt.datetime(2022, 5, 30)
end_date =  dt.datetime.today()
# end_date =  dt.datetime(2022, 5, 22)
interval = '12h' 

base_currency = 'BUSD'

coins = ['BNB']

# Downloading the CSV files for the coins
for coin in coins:
  get_binance_bars(coin, base_currency, interval, start_date, end_date)

In [635]:
# 12 hours data.........
data_bnb_12h_live= pd.read_csv('BNB.csv')
data_bnb_12h_live = data_bnb_12h_live.rename(columns={'Unnamed: 0': 'datetime' })
data_bnb_12h_live.set_index('datetime', inplace= True)

# print( "live  data", data_bnb_12h.shape)
data_bnb_12h_live = data_bnb_12h_live.iloc[2:] 
data_bnb_12h_live

,open,high,low,close,volume
datetime,,,,,
2022-05-31 05:30:00+05:30,321.6,324.5,315.7,319.8,110841.822


In [636]:

past_12h = pd.read_csv( 'data_bnb_12h.csv')
past_12h.set_index('datetime', inplace= True)
past_12h 

,open,high,low,close,volume
datetime,,,,,
2019-09-19 00:00:00,21.2021,21.2058,20.7415,20.9594,25.810
2019-09-19 12:00:00,20.9454,21.5998,20.8856,21.5998,2390.910
2019-09-20 00:00:00,21.4918,21.5380,21.1116,21.4251,931.330
2019-09-20 12:00:00,21.4251,21.4251,20.9728,21.2280,83.480
2019-09-21 00:00:00,21.3134,21.4138,21.0140,21.1000,13.060
...,...,...,...,...,...
2022-05-29 00:00:00,307.1000,307.8000,298.3000,302.5000,119025.793
2022-05-29 12:00:00,302.6000,307.5000,301.4000,305.6000,113564.508
2022-05-30 00:00:00,305.6000,321.3000,303.6000,317.0000,224312.977


In [637]:
frames4 = [past_12h,data_bnb_12h_live]
data_bnb_12h  = pd.concat(frames4)
print(data_bnb_12h.tail( ) )


                            open   high    low  close      volume
datetime                                                         
2022-05-29 12:00:00        302.6  307.5  301.4  305.6  113564.508
2022-05-30 00:00:00        305.6  321.3  303.6  317.0  224312.977
2022-05-30 12:00:00        317.0  323.8  311.5  321.5  200876.847
2022-05-31 00:00:00        321.6  324.5  317.2  319.0   67822.382
2022-05-31 05:30:00+05:30  321.6  324.5  315.7  319.8  110841.822


In [638]:
def get_binance_bars(coin, base_currency, interval, startTime, endTime):
    ''' 
    download the data of the coins and store as .csv file 
    '''
     
    symbol = str(coin+base_currency)
    
    # In case of HTTP 5xx Error, or data is not fetched from the server, Try changing 'api1' in the URL to 'api2' or 'api3'

    url = "https://api1.binance.com/api/v3/klines"
 
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '4000'
 
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}
    
    data = json.loads(requests.get(url, params = req_params).text)

    if isinstance(data , dict):
      print(f"Returned Invalid Code Error for Coin {coin}")
      return None
    
    else:
      if not data:
        print(f"Did not receieve OK response from Coinbase API for Coin {coin}")
        return None
      
      else:
        df = pd.DataFrame(data)

        df = df.iloc[:, 0:6]
        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume' ]
    
        df.open      = df.open.astype("float")
        df.high      = df.high.astype("float")
        df.low       = df.low.astype("float")
        df.close     = df.close.astype("float")
        df.volume    = df.volume.astype("float")
         

        df.set_index('datetime', inplace= True)
 
        df.index = [pd.Timestamp(x/1000, unit='s', tz= 'Asia/Kolkata') for x in df.index]
        df = df.astype('float')

        # df['datetime'] = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
        # df.set_index('datetime', inplace= True)  
        

        df.to_csv(f'{coin}.csv')

In [639]:
# KEY PARAMETERS
# end_date =  dt.datetime.today()
# start_date = end_date - dt.timedelta(days =990)
# interval = '1d'

start_date = dt.datetime(2022, 5, 30)
end_date =  dt.datetime.today()
# end_date =  dt.datetime(2022, 5, 22)
interval  = '1d'

base_currency = 'BUSD'

coins = ['BNB']

# Downloading the CSV files for the coins
for coin in coins:
  get_binance_bars(coin, base_currency, interval, start_date, end_date)

In [640]:
# 1 day data...............
data_bnb_1d_live = pd.read_csv('BNB.csv')
data_bnb_1d_live = data_bnb_1d_live.rename(columns={'Unnamed: 0': 'datetime' })
data_bnb_1d_live.set_index('datetime', inplace= True)

# print( "live  data", data_bnb_1d.shape)
data_bnb_1d_live = data_bnb_1d_live.iloc[2:]
data_bnb_1d_live

,open,high,low,close,volume
datetime,,,,,


In [641]:

past_1d = pd.read_csv( 'data_bnb_1d.csv')
past_1d.set_index('datetime', inplace= True)
past_1d 

,open,high,low,close,volume
datetime,,,,,
2019-09-19,21.2021,21.5998,20.7415,21.5998,2416.720
2019-09-20,21.4918,21.5380,20.9728,21.2280,1014.810
2019-09-21,21.3134,21.4138,20.8128,20.8755,35.590
2019-09-22,20.7415,20.7415,20.0146,20.4181,198.810
2019-09-23,20.3258,20.3258,19.2092,19.3267,804.230
...,...,...,...,...,...
2022-05-27,302.6000,309.6000,290.3000,301.2000,554306.137
2022-05-28,301.2000,310.1000,296.2000,307.2000,246188.010
2022-05-29,307.1000,307.8000,298.3000,305.6000,232590.301


In [642]:
frames5= [past_1d,data_bnb_1d_live]
data_bnb_1d  = pd.concat(frames5)
print(data_bnb_1d  )


                open      high       low     close      volume
datetime                                                      
2019-09-19   21.2021   21.5998   20.7415   21.5998    2416.720
2019-09-20   21.4918   21.5380   20.9728   21.2280    1014.810
2019-09-21   21.3134   21.4138   20.8128   20.8755      35.590
2019-09-22   20.7415   20.7415   20.0146   20.4181     198.810
2019-09-23   20.3258   20.3258   19.2092   19.3267     804.230
...              ...       ...       ...       ...         ...
2022-05-27  302.6000  309.6000  290.3000  301.2000  554306.137
2022-05-28  301.2000  310.1000  296.2000  307.2000  246188.010
2022-05-29  307.1000  307.8000  298.3000  305.6000  232590.301
2022-05-30  305.6000  323.8000  303.6000  321.5000  425189.824
2022-05-31  321.6000  324.5000  317.2000  319.4000   68088.528

[986 rows x 5 columns]


In [643]:
# df2.info()
da = data_bnb_1h.iloc[-1].name
# type(da)
da

'2022-05-31 14:30:00+05:30'

In [644]:
from datetime import datetime

da  = datetime.fromisoformat(da)
type(da)

datetime.datetime

In [645]:
#  dates of predictions
from datetime import datetime
def days( ):
  d1 = da
  d2 = da + dt.timedelta(hours = 1)
  a = dt.datetime.strftime( d1,"%Y-%m-%d %H:%M:%S")
  h_1 =  dt.datetime.strftime (d2, "%Y-%m-%d %H:%M:%S")
  h_2 =  dt.datetime.strftime (da + dt.timedelta(hours = 2), "%Y-%m-%d %H:%M:%S")
  h_4 =  dt.datetime.strftime (da + dt.timedelta(hours = 4), "%Y-%m-%d %H:%M:%S")
  h_12 =  dt.datetime.strftime (da + dt.timedelta(hours = 12), "%Y-%m-%d %H:%M:%S")
  d_1 =  dt.datetime.strftime (da + dt.timedelta(days = 1), "%Y-%m-%d %H:%M:%S")
   
  return  a,h_1,h_2,h_4,h_12,d_1
a,h_1,h_2 ,h_4,h_12,d_1= days() 
a,h_1,h_2,h_4,h_12,d_1

('2022-05-31 14:30:00',
 '2022-05-31 15:30:00',
 '2022-05-31 16:30:00',
 '2022-05-31 18:30:00',
 '2022-06-01 02:30:00',
 '2022-06-01 14:30:00')

In [646]:
#  shift close column for next future prediction
def shift_col(dataf,coln):
  num = 1
  if(num>0):

    dataf[ coln + '_pred']= dataf[[coln]].shift(num) 
     
  return dataf 
df_close_1h = shift_col(data_bnb_1h,'close')
df_close_2h = shift_col(data_bnb_2h,'close')
df_close_4h = shift_col(data_bnb_4h,'close')
df_close_12h = shift_col(data_bnb_12h,'close')
df_close_1d = shift_col(data_bnb_1d,'close')
 
 
 

In [647]:
#  removing NAN values
df_close_1h = df_close_1h.dropna()
df_close_2h = df_close_2h.dropna()
df_close_4h = df_close_4h.dropna()
df_close_12h = df_close_12h.dropna()
df_close_1d = df_close_1d.dropna()


In [648]:
from sklearn.model_selection import train_test_split 

In [649]:
# shape of all data frame
data_bnb_1h.shape,data_bnb_2h.shape,data_bnb_4h.shape,data_bnb_12h.shape,data_bnb_1d.shape

((23612, 6), (11805, 6), (5913, 6), (1972, 6), (986, 6))

In [650]:
#  split data into train and test data
def split_data(data_f,x1,x2,x3,x4,y1):
  X= np.array(data_f[[ x1,x2,x3,x4]])
  Y= np.array(data_f[[y1] ] )
  x_train, x_test, y_train, y_test = train_test_split(X , Y , shuffle=False)
  print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
  return x_train, x_test, y_train, y_test

x_train_df_close_1h, x_test_df_close_1h, y_train_df_close_1h, y_test_df_close_1h  = split_data(df_close_1h,'open','high','low','volume','close_pred')
x_train_df_close_2h, x_test_df_close_2h, y_train_df_close_2h, y_test_df_close_2h  = split_data(df_close_2h,'open','high','low','volume','close_pred')
x_train_df_close_4h, x_test_df_close_4h, y_train_df_close_4h, y_test_df_close_4h  = split_data(df_close_4h,'open','high','low','volume','close_pred')
x_train_df_close_12h, x_test_df_close_12h, y_train_df_close_12h, y_test_df_close_12h  = split_data(df_close_12h,'open','high','low','volume','close_pred')
x_train_df_close_1d, x_test_df_close_1d, y_train_df_close_1d, y_test_df_close_1d  = split_data(df_close_1d,'open','high','low','volume','close_pred')


(17708, 4) (5903, 4) (17708, 1) (5903, 1)
(8853, 4) (2951, 4) (8853, 1) (2951, 1)
(4434, 4) (1478, 4) (4434, 1) (1478, 1)
(1478, 4) (493, 4) (1478, 1) (493, 1)
(738, 4) (247, 4) (738, 1) (247, 1)


In [651]:
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error, mean_absolute_error

In [652]:
from sklearn.ensemble import RandomForestRegressor

In [653]:
#  model building...........random forest regressor model
def rfr_model(x_train ,y_train ,y_test ,x_test):
  regr = RandomForestRegressor(max_depth=11)
  regr.fit(x_train,y_train)
  y_pred_rfr= regr.predict(x_test)
  
  mean_absolute_percentage_error_RFR = mean_absolute_percentage_error(y_test, y_pred_rfr)
  mean_absolute_error_RFR = mean_absolute_error(y_test, y_pred_rfr)
  mean_squared_error_RFR = mean_squared_error(y_test,y_pred_rfr)
  return y_pred_rfr,mean_absolute_percentage_error_RFR,mean_absolute_error_RFR,mean_squared_error_RFR,regr,y_test

y_pred_rfr_df_close_1h,mape_rfr_df_close_1h,mae_rfr_df_close_1h,mse_rfr_df_close_1h,model_df_close_1h,y_test_close_1h = rfr_model(x_train_df_close_1h, y_train_df_close_1h, y_test_df_close_1h, x_test_df_close_1h)
y_pred_rfr_df_close_2h,mape_rfr_df_close_2h,mae_rfr_df_close_2h,mse_rfr_df_close_2h,model_df_close_2h,y_test_close_2h = rfr_model(x_train_df_close_2h, y_train_df_close_2h, y_test_df_close_2h, x_test_df_close_2h)
y_pred_rfr_df_close_4h,mape_rfr_df_close_4h,mae_rfr_df_close_4h,mse_rfr_df_close_4h,model_df_close_4h,y_test_close_4h = rfr_model(x_train_df_close_4h, y_train_df_close_4h, y_test_df_close_4h, x_test_df_close_4h)
y_pred_rfr_df_close_12h,mape_rfr_df_close_12h,mae_rfr_df_close_12h,mse_rfr_df_close_12h,model_df_close_12h,y_test_close_12h = rfr_model(x_train_df_close_12h, y_train_df_close_12h, y_test_df_close_12h, x_test_df_close_12h)
y_pred_rfr_df_close_1d,mape_rfr_df_close_1d,mae_rfr_df_close_1d,mse_rfr_df_close_1d,model_df_close_1d,y_test_close_1d = rfr_model(x_train_df_close_1d, y_train_df_close_1d, y_test_df_close_1d, x_test_df_close_1d)

# mape_rfr_df_close,mae_rfr_df_close,mse_rfr_df_close, mape_rfr_df_low,mae_rfr_df_low,mse_rfr_df_low
# print( mape_rfr_df2,mae_rfr_df2,mse_rfr_df2 ) 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  af

In [654]:
print("accuracy of close_1h : ",model_df_close_1h.score( x_test_df_close_1h,  y_test_df_close_1h))
print("accuracy of close_2h : ",model_df_close_2h.score( x_test_df_close_2h,  y_test_df_close_2h))
print("accuracy of close_4h : ",model_df_close_4h.score( x_test_df_close_4h,  y_test_df_close_4h))
print("accuracy of close_12h : ",model_df_close_12h.score( x_test_df_close_12h,  y_test_df_close_12h))
print("accuracy of close_1d : ",model_df_close_1d.score( x_test_df_close_1d,  y_test_df_close_1d))

accuracy of close_1h :  0.999995297830274
accuracy of close_2h :  0.9999859914781003
accuracy of close_4h :  0.9999451305469705
accuracy of close_12h :  0.9996983696083608
accuracy of close_1d :  0.9991484540779171


In [655]:
#  access last row of data frame  to predict next price
def loc_data_df(loc_data,a,b,c,d):
  val1 = loc_data[ a].iloc[-1]
  val2 = loc_data[b].iloc[-1]
  val3 = loc_data[c].iloc[-1]
  val4 = loc_data[d].iloc[-1]
  
  return val1,val2,val3,val4
df2_d_close_1h = loc_data_df(df_close_1h,'open','low','high','volume')
df2_d_close_2h = loc_data_df(df_close_2h,'open','low','high','volume')
df2_d_close_4h = loc_data_df(df_close_4h,'open','low','high','volume')
df2_d_close_12h = loc_data_df(df_close_12h,'open','low','high','volume')
df2_d_close_1d = loc_data_df(df_close_1d,'open','low','high','volume')

 

In [656]:
# predict close price of all intervlas 
def price_pred(model,val):
  
  pred_price= model.predict(np.array([val])) 
  return pred_price
  
df2_pred_close_1h = price_pred(model_df_close_1h,df2_d_close_1h) 
df2_pred_close_2h = price_pred(model_df_close_2h,df2_d_close_2h) 
df2_pred_close_4h = price_pred(model_df_close_4h,df2_d_close_4h) 
df2_pred_close_12h = price_pred(model_df_close_12h,df2_d_close_12h) 
df2_pred_close_1d = price_pred(model_df_close_1d,df2_d_close_1d) 


In [657]:
print("Price Prediction Of BNB")
def pred_table():
  # time = b
  
  pred_table = pd.DataFrame( )
  predicted_price = [df2_pred_close_1h.round(2),df2_pred_close_2h.round(2), df2_pred_close_4h.round(2),df2_pred_close_12h.round(2),df2_pred_close_1d.round(2)]
  interval =['close_1h','close_2h','close_4h','close_12h','close_1d']
  pred_time = [h_1,h_2,h_4,h_12,d_1]
 
  pred_table['Time'] = pred_time
  pred_table['Interval']=interval
  pred_table['Predicted_close_price']=predicted_price
   
  pred_table.set_index('Time', inplace= True)
  
  return pred_table
price = pred_table()
price

Price Prediction Of BNB


,Interval,Predicted_close_price
Time,,
2022-05-31 15:30:00,close_1h,[319.84]
2022-05-31 16:30:00,close_2h,[321.58]
2022-05-31 18:30:00,close_4h,[318.22]
2022-06-01 02:30:00,close_12h,[321.65]
2022-06-01 14:30:00,close_1d,[320.83]


In [660]:
# save the result in txt file
with open("randomfile.txt", "a") as external_file:
     
    print(price, file=external_file)
    external_file.close()